In [1]:
import sys
sys.path.append('../')

from data_partitioner import *
from parallel_tagging import *
from biberplus.tagger import load_config, load_pipeline, calculate_tag_frequencies
from biberplus.reducer import encode_text
from biberplus.tagger.tagger_utils import load_tokenizer

INFO:numexpr.utils:Note: detected 160 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 160 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
author_subreddit_counts = {}

## Partition the file into 100 parts

In [3]:
input_file = '/shared/4/datasets/long-reddit/RC_2022-01.gz'
output_directory = '/shared/3/projects/hiatus/tagged_data/partitions/reddit/'

In [4]:
partition_file(input_file, output_directory, chunks=100)

INFO:root:7282008 lines in file
INFO:root:Chunk size: 72820 lines


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte

## Tag each partition with 1 CPU 

Set nice value low so we don't hog the server

**Tagger config**

In [2]:
config = load_config()
config.update({'use_gpu': False, 
               'biber': True,
               'binary_tags': True, 
               'function_words': True,/shared/4/datasets/long-reddit/RC_2022-01.gz
               'token_normalization': 100})
tokenizer = load_tokenizer(use_gpu=False)
config

{'biber': True,
 'function_words': True,
 'binary_tags': True,
 'function_words_list': [],
 'token_normalization': 100,
 'use_gpu': False,
 'show_progress': False,
 'n_processes': 1,
 'processing_size': 10000,
 'return_text': False,
 'drop_last_batch_pct': 0.5}

In [3]:
tag_partitions(config,
               input_directory='/shared/4/datasets/long-reddit/',
               output_directory='/shared/3/projects/hiatus/tagged_data/long-reddit/',
               num_workers=2,
               post_counts=author_subreddit_counts
               )

Tagging file /shared/3/projects/hiatus/partitions/reddit/partition-1.jsonl
Tagging file /shared/3/projects/hiatus/partitions/reddit/partition-14.jsonl




KeyboardInterrupt: 

In [ ]:
# Write the post counts to a TSV file
write_counts_to_tsv(author_subreddit_counts, "/shared/3/projects/hiatus/tagged_data/long-reddit/author_counts.tsv")

## Join the partitioned files

In [ ]:
join_tagged_files(input_directory='/shared/2/projects/hiatus/partitions/reddit/',
                  ouput_file='/shared/3/projects/hiatus/reddit/corpus_tagged.jsonl')

## Delete the partitions directory

In [ ]:
delete_partitioned_files('/shared/2/projects/hiatus/partitions/reddit/')

In [ ]:
delete_partitioned_files('/shared/3/projects/hiatus/partitions/reddit/')